In [1]:
!nvidia-smi || true
import torch, platform
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

Wed Aug 27 02:22:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             41W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
%%bash
pip -q install -U open_clip_torch pillow pyarrow pandas tqdm
python - <<'PY'
import torch
try:
    torch.set_float32_matmul_precision('high')
except Exception:
    pass
print('Torch:', torch.__version__)
print('open_clip installed ✓')
PY

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 129.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
Torch: 2.8.0+cu126
open_clip installed ✓


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
ITEMS = "/content/drive/MyDrive/MMREC/data/items.parquet"  #@param {type:"string"}
INTERACTIONS = "/content/drive/MyDrive/MMREC/data/interactions.parquet"  #@param {type:"string"}
SCRIPT = "/content/drive/MyDrive/MMREC/clip_v3.py"  #@param {type:"string"}
OUT_DIR = "/content/drive/MyDrive/MMREC/data/features"  #@param {type:"string"}
IMG_DIR = "/content/drive/MyDrive/MMREC/data/images"  #@param {type:"string"}
ONLY_INTERACTED = True  #@param {type:"boolean"}
LIMIT_ITEMS = 0         #@param {type:"integer"}
TXT_BS = 512            #@param {type:"integer"}
IMG_BS = 128            #@param {type:"integer"}
WORKERS = 6             #@param {type:"integer"}
CLIP_MODEL = "ViT-B-32" #@param {type:"string"}
CLIP_PRETRAINED = "openai" #@param {type:"string"}
DEVICE = "cuda"         #@param ["cuda","cpu"]
USE_EXISTING_ONLY = True

import os
for p in [OUT_DIR, IMG_DIR]:
    os.makedirs(p, exist_ok=True)
print('Paths set.')

Paths set.


In [ ]:
cmd = f"""
python -u "{SCRIPT}" \
  --items "{ITEMS}" \
  --interactions "{INTERACTIONS}" \
  --out "{OUT_DIR}" \
  --img_dir "{IMG_DIR}" \
  --only-interacted --workers {WORKERS} \
  --txt-bs {TXT_BS} --img-bs {IMG_BS} \
  --clip-model "{CLIP_MODEL}" --clip-pretrained "{CLIP_PRETRAINED}" \
  --device {DEVICE} {"--limit-items "+str(int(LIMIT_ITEMS)) if LIMIT_ITEMS and int(LIMIT_ITEMS)>0 else ""} \
  {"--use-existing-only" if USE_EXISTING_ONLY else ""}
"""
print(cmd)
get_ipython().system(cmd)



python -u "/content/drive/MyDrive/MMREC/clip_v3.py"   --items "/content/drive/MyDrive/MMREC/data/items.parquet"   --interactions "/content/drive/MyDrive/MMREC/data/interactions.parquet"   --out "/content/drive/MyDrive/MMREC/data/features"   --img_dir "/content/drive/MyDrive/MMREC/data/images"   --only-interacted --workers 6   --txt-bs 512 --img-bs 128   --clip-model "ViT-B-32" --clip-pretrained "openai"   --device cuda    --use-existing-only

[Stats] items: 825869 | images ready: 30812 (3.73%) | will encode images only for existing files.
open_clip_model.safetensors: 100% 605M/605M [00:01<00:00, 438MB/s]
CLIP text: 100% 825869/825869 [03:23<00:00, 4067.64txt/s]
CLIP image: 100% 30812/30812 [1:07:59<00:00,  7.55img/s]
Saved: /content/drive/MyDrive/MMREC/data/features/item_features_clip.parquet and /content/drive/MyDrive/MMREC/data/features/user_stats.parquet
items encoded: 825869  | failures: 0


In [ ]:
import os, re, numpy as np, pandas as pd

FEATS_IN = "/content/drive/MyDrive/MMREC/data/features/item_features_clip.parquet" 
print("mtime:", os.path.getmtime(FEATS_IN), "size(MB):", os.path.getsize(FEATS_IN)/1024/1024)

feats = pd.read_parquet(FEATS_IN)
img_cols = [c for c in feats.columns if re.match(r"^(clip_img|img_emb)_\d+$", c)]
print("img cols:", img_cols)
img_col = img_cols[0]

def nonzero(a):
    a = np.asarray(a, dtype=np.float32).ravel()
    return a.size>0 and np.any(a!=0)

n_all = feats["item_id"].nunique()
n_nz  = feats[img_col].apply(nonzero).sum()
print(f"items total={n_all:,} | nonzero image emb={n_nz:,} ({n_nz/n_all:.2%})")
feats.head(2)


mtime: 1756266130.0 size(MB): 794.308177947998
img cols: ['clip_img_512']
items total=825,869 | nonzero image emb=30,812 (3.73%)


,item_id,clip_txt_512,clip_img_512,price_log1p,has_image,image_path
0,B08BHN9PK5,"[-0.0016107559204101562, -0.048614501953125, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,False,None
1,B08R39MRDW,"[0.028533935546875, 0.0176239013671875, -0.003...","[0.0161285400390625, 0.0037555694580078125, 0....",0.0,True,/content/drive/MyDrive/MMREC/data/images/B08R3...
